In [4]:
import numpy as np
import pandas as pd
from check_solution import check_solution

df = pd.read_csv('../data/bank_data.csv')

# find the list and value of the best route for top n banks
def robbed_list(df, n):
    
    # create an array of the top n banks that have the best money to time ratio
    df['weight'] = df['money']/df['time (hr)']
    best_list = df.sort_values('weight', ascending=False).id.head(n).reset_index(drop=True).to_numpy()
    
    # get the bank id with the best weight
    currentid = df[['weight']].idxmax()[0] 
    
    robbed = [currentid]
    money = 0
    
    # index of values will be the bank id
    coordinates = df[['x_coordinate', 'y_coordinate']].values # index of values will be the bank id
    
    # initialize the time calculations
    time_left = 24.0
    time_left -= df['time (hr)'][currentid]
    time_to_escape = (np.linalg.norm(coordinates[currentid] - (0,0)) / 30) # distance to center over 30 to get hours
    
    while time_left >= time_to_escape:
        
        # returns best_list array with already robbed banks removed
        best_list = np.setdiff1d(best_list, robbed, assume_unique=True)
        
        # create an array of distances to all other banks with higher value banks shortening the distance
        node_weights = df.iloc[best_list].weight.to_numpy()
        distances = np.linalg.norm(coordinates[best_list] - coordinates[currentid], axis=1)
        distance_weights = distances / node_weights
        
        # get the lowest weight distance and select the bank at that location
        next_bank = best_list[distance_weights.argmin()]
        
        # update time
        time_to_rob = df['time (hr)'][next_bank]
        time_to_travel = (np.linalg.norm(coordinates[currentid] - coordinates[next_bank]) / 30)
        time_left = time_left - time_to_rob - time_to_travel
        time_to_escape = (np.linalg.norm(coordinates[next_bank] - (0,0)) / 30)
        
        # add next bank to robbed list and take money from current bank
        robbed.append(next_bank)
        money += df.money[currentid]
        
        currentid = next_bank
        
    return robbed[:-1], money 
        # rounding errors cause issues with submitting full list so just remove last element


# function to determine how many of the top weighted banks to include in the list
def best_robbed_list(df):
    
    best_robbed_dict = {}
    
    # get lists and money value for different sizes of best_list
    for i in range(270,500):
        best_robbed_dict.update({i : robbed_list(df,i)[1]})
    
    # find the best_list size that returns the highest value
    vals = best_robbed_dict.values()
    mx = max(vals)
    for key, val in best_robbed_dict.items():
        if val == mx:
            best_id = key

    return robbed_list(df, best_id)[0]


bank_id_list = best_robbed_list(df)

In [5]:
check_solution(bank_id_list, df, speed=30.)

Time Remaining: 0.046716781529124694


14119600.0